Hui Yu's answers to these three questions.

What was your biggest challenge in this project? (regarding writing code and not only)

The biggest challenge is to understand K-means. It took me quite a while to read through the file and understand the concept. I was quite confused at begining why the accuracy number is different each time I ran. I will remember to set up random_state to make my results reproducible. 

What did you learn while working on this project? (regarding writing code and not only)

I noticed that the KMeans method is acturally very common used. In my own single cell analysis, the k-mean is widely used to cluster cells and generate UMAP plot.I was not fully aware of it until I read through the kmean document from this project. 
When seperating train and test datasets, I was thinking to manually seperate half dataset for train dataset and half for test dataset, however, my group member used  sklearn.model_selection train_test_split function. Very smart. 


If you had more time on the project what other question(s) would you like to answer? (at least one question is required)

I would like to compare the three accuracy numbers and investigate why I got the highest accuracy score at the train dataset and why the accuracy scores have this trend in the three tests? What can be possibly done to improve the accuracy score (from both sample collections and fitting to approprate model point of views)?

----------------------------------------------------------------------------------------------------------------

Keagan Moo's answers to the three questions:

What was your biggest challenge in this project? (regarding writing code and not only)

The biggest coding challenge was wrapping my head around the structure of the GEOParse object and making sure I was getting the correct pieces out of it. 

What did you learn while working on this project? (regarding writing code and not only)

I learned that sklearn designates its own internal state for reproducibility rather than depending on the environment for the seed. That had not been my experience with similar tools in R or Python so it was good to know. 

If you had more time on the project what other question(s) would you like to answer? (at least one question is required)

I'd like to ask if a consensus model of some kind could be generated by training the model with thirds of the data and combining the results in some meaningful way. Then of course I'd like to know how that stacks up against other methods.


Maya Bose question responses
1. My biggest challenge for the project was understanding how the part of the project I was working on fit into the bigger picture of the assignment. I had never worked on coding projects in groups before, and so learning how to understand what other people's code was doing and how to work with it was very new to me. 

2. I learned that scikit learn has a function for dividing a dataset into a testing set and a training set called test_train_split. 

3. If I had more time to work of the project, I would like to explore what parameters may optimize the accuracy of the KMeans clustering, or explore if there are other classification methods that may fit the situation better. 

In [70]:
import GEOparse
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split
%matplotlib inline
import matplotlib.pyplot as plt
random_seed = 3

# 1 - Use the GEOparse module to load the data

In [ ]:
rawDataGSE10245 = GEOparse.get_GEO(geo="GSE10245", destdir="./")

/home/mbose/miniconda3/lib/python3.9/site-packages/GEOparse/GEOparse.py:545: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  table_data = parse_table_data(gpl_soft)


In [ ]:
slowpoke = True
if slowpoke:
    print(type(rawDataGSE10245))
    print(rawDataGSE10245.metadata['summary'][0])
    print("\nSample GSM258551")
    for metaIndex in rawDataGSE10245.gsms.get("GSM258551").metadata.items():
        print(str(metaIndex))
    print("\nPlatform GPL570")
    for metaIndex in rawDataGSE10245.gpls.get('GPL570').metadata.items():
        print(str(metaIndex))

In [ ]:
rawDataGSE10245.gsms.get('GSM258551').table

In [ ]:
# So to get a useable dataframe we will join each sample together iteratively. 
# Another, possibly faster, method would be to check if all of the rows are the same in each file, but this
# solution is more generalizable in the case where some rows might be filtered out of some samples
dfList = list()
# First we will iterate over all of the samples avialable in the object
for sampleIndex in rawDataGSE10245.gsms:
    # Get the current sample table from the larger object
    sampleData = rawDataGSE10245.gsms.get(sampleIndex).table
    # get the rownames for assignment to the dataframe
    rowNames = sampleData['ID_REF']
    # get the actual data for assignment to the dataframe
    colData = sampleData['VALUE']
    # initialize a temporary dataframe just for the sample and include that samples data with
    # the sample number as the column head
    sampleDataFrame = pd.DataFrame({sampleIndex:colData})
    # reattatch the row names as the index for this temporary dataframe to perserve this sample's
    # reference structure on the off chance they are different between samples
    sampleDataFrame.index = rowNames
    # add this temporary dataframe to the list of all samples' temporary dataframes
    dfList.append(sampleDataFrame)
# Join all temporary dataframes together by index in one step starting with the first and then 
# joining all others in the list.
rawDataFrameGSE10245 = dfList[0].join(dfList[1:])

In [ ]:
print(rawDataFrameGSE10245)
rawDataFrameGSE10245.to_csv(r'BiorawDataFrameGSE10245.csv')

In [ ]:
df= rawDataFrameGSE10245.T

# 2 - Use the sklearn.cluster module to do the clustering

In [ ]:
# Then we need to generate a K-means object using the appropriate number (k) of means
# Thanks to Hui for identifying the need for the "random_state" parameter to reproduce results.
# When actually using this method you would remove this parameter so as not to risk selecting results.
kmeansInit = KMeans(n_clusters = 2,random_state=random_seed)
# We give the kmeans object the data in question to generate the cluster points
kmeansInit.fit(df)
# Finally we ask it to predict the data used to train it and generate the cluster codes for each sample
GSE10245Means = kmeansInit.predict(df)
# Let's attach these values to a Series labeled with the sample names for the benefit of those who follow
GSE10245ClusterCodes = pd.Series(GSE10245Means, index = df.index)
GSE10245ClusterCodes.to_csv(r'clusterCodesGSE10245.csv')

In [ ]:
GSE10245Means

In [ ]:
df.head()

# 3 - Create a DataFrame that contains the 58 sample labels as one column, the cluster number as another column and the subtype given in the dataset (adenocarcinoma or squamous cell carcinoma) as another column

In [ ]:
## this code is adapted from Maya Bose's original code
rawDataGSE10245 = GEOparse.get_GEO(filepath="GSE10245_family.soft.gz", silent=True)
tumor_subtypes = list()
for sample in rawDataGSE10245.gsms:
    tumor_subtype = rawDataGSE10245.gsms.get(sample).metadata['characteristics_ch1'][0].split(":")[1].strip()
    if tumor_subtype == "adenocarcinoma":
        tumor_subtypes.append("AD")
    elif tumor_subtype == "squamous cell carcinoma":
        tumor_subtypes.append("SCC")
    else:
        tumor_subtypes.append("UNKNOWN")
df = df.assign(tumor_subtype = tumor_subtypes)
GSE10245ClusterCodes = GSE10245ClusterCodes.to_frame().assign(tumor_subtype = tumor_subtypes)

In [ ]:
GSE10245ClusterCodes

In [ ]:
## for problem 3, create a dataframe contains 58 sampes labels, cluster number and subtypes
GSE10245ClusterCodes.columns =['Cluster_Number','Tumor_Subtype']
GSE10245ClusterCodes.index.names=["Sample_Name"]
GSE10245ClusterCodes.to_csv(r'cluster_tumor_type_infor.csv') 
## problem 3 is done

In [ ]:
GSE10245ClusterCodes.head()

In [ ]:
# for problem 4, Compute and display the accuracy of this clustering.
## first set up "SCC" = 1 and "AD" = 0
GSE10245Means
GSE10245Means = np.where(GSE10245Means == 0, "AD", "SCC")

In [ ]:
GSE10245Means

In [ ]:
## want to check the overal accuracy by adding the predicted subtype in the dataframe
df_2 = GSE10245ClusterCodes.assign(predicted=GSE10245Means)

In [ ]:
df_2.head()

# 4 - Compute and display the accuracy of this clustering. 

In [ ]:
complete_dataset_acc = accuracy_score(tumor_subtypes, GSE10245Means)

In [ ]:
print(f"The accuracy score when testing on all of the data is {complete_dataset_acc}.")

# 5 - Fit the model on half of the data (balanced: 20 AD and 9 SCC) and assign a label to the clusters (AD or SCC), then use the model to predict the cluster/label for the other half of the data.

Lets partition our df into AD and SCC samples. randomly choose 20 AD samples and 9 SCC sample. We will designate these the "test" set. The remainder of the samples will be the "validation" set.

In [ ]:
ad = df[df["tumor_subtype"] == "AD"]
scc = df[df["tumor_subtype"] == "SCC"]
print(f"There are {ad.shape[0]} AD samples and {scc.shape[0]} SCC samples")

We want to use half of the AD samples as our "training" set, and the other half as our "testing" set. Ditto for the SCC samples. We can use the sklearn.model_selection train_test_split function to do this. test_size indicates the ratio of training samples to testing samples; in this case, the ratio is 50:50 (half).

In [ ]:
train_ad, test_ad = train_test_split(ad, test_size = 0.5, random_state = random_seed)
train_scc, test_scc = train_test_split(scc, test_size = 0.5, random_state = random_seed)

Let's combine our "training" set for AD samples with our "training" set for SCC samples into a single training set. Do the same to create a single testing set. We also want to remove the tumor subtype annotations, so that we can then use the model to predict these annotations, but we will also save the labels in a seperate variable for testing the accuracy of the model later.

In [ ]:
# Combine AD training samples with SCC training samples to create a single training DF
train = pd.concat([train_ad, train_scc])
# Save the tumor subtype labels for accuracy testing later
train_labels = train["tumor_subtype"].values
# drop the tumor subtype labels to use the DF in clustering
train = train.drop("tumor_subtype", axis = 1)

# Combine AD testing samples with SCC testing samples to create a single testing DF
test = pd.concat([test_ad, test_scc])
# Save the tumor subtype labels for accuracy testing later
test_labels = test["tumor_subtype"].values
# drop the tumor subtype labels to use the DF in clustering
test = test.drop("tumor_subtype", axis = 1)

In [ ]:
# Check out what one of these DFs looks like to make sure everything's good
train.head()

Now, we can train the model using the training set

In [ ]:
# Train the model
kmeansInit.fit(train)

# Use the trained model to predict tumor states in the training set
train_kmeans = kmeansInit.predict(train)
# Use the trained model to predict tumor states in the testing set
test_kmeans = kmeansInit.predict(test)

In [ ]:
# Check out what the clusters look like
train_kmeans 

We will map the 0 cluster to AD and the 1 cluster to SSC.

In [ ]:
train_kmeans = np.where(train_kmeans == 0, "AD", "SCC")
test_kmeans = np.where(test_kmeans == 0, "AD", "SCC")

In [ ]:
train_kmeans

Neat. Step 5 completed! Now for step 6 (computing and displaying the accuracy of the model for the training and testing data). We can do this by using sklearn.metrics functions accuracy_score to compare the known labels (stored in variables earlier) to the cluster predictions.
Accuracy score for the training data set:

# 7 - Compute and display the accuracy of the model for the prediction results (testing data).

In [ ]:
print(f"The accuracy score when training on half of the data is {accuracy_score(train_labels, train_kmeans)}.")
training_acc = accuracy_score(train_labels, train_kmeans)

Accuracy score for the testing data set:

In [ ]:
print(f"The accuracy score when testing on half of the data is {accuracy_score(test_labels, test_kmeans)}.")
testing_acc = accuracy_score(test_labels, train_kmeans)

# 8 - Plot the 3 accuracy values: for the model when training on all the data, for the model when trining on half of the data, for the model when testing on the half of the data (test data)

The three numbers we need to plot are 0.9310344827586207 for  complete dataset, 0.9310344827586207 for training dataset, and 0.896551724137931 for testing dataset


In [ ]:
## create a bar chart and 
## assign the name for each bar 
names = ['complete_dataset','training_dataset','testing_dataset']

In [ ]:
## assigne the accuracy_score to each bar 
values = [complete_dataset_acc, training_acc, testing_acc]

In [ ]:
plt.bar(names, values)
plt.ylabel('accuracy_score')
plt.title('Plot the 3 accuracy values')

In [ ]:
plt.savefig('plot_3_accuracy_values.png')